In [8]:
import pandas as pd
import sys
import requests as r

sys.path.append('../helper_functions')
import github_utils as gh
import tokenlist_utils as tl
import duneapi_utils as dapi

In [9]:
gh_url = gh.get_file_url_from_github('ethereum-optimism','ethereum-optimism.github.io','','optimism.tokenlist.json')
dune_list_query_id = 2420477 #https://dune.com/queries/2420477

In [10]:
# Load the JSON data from the URL
res = r.get(gh_url)
data = res.json()


df = tl.generate_table_from_tokenlist(data)
df = df[df['chainId'] ==10] #Get OP Version
display(df)

,chainId,address,name,symbol,decimals,extensions_optimismBridgeAddress,extensions_opListId,extensions_opTokenId
1,10,0xe0BB0D3DE8c10976511e5030cA403dBf4c25165B,0xBitcoin,0xBTC,8,0x4200000000000000000000000000000000000010,extended,0xBTC
3,10,0x76FB31fb4af56892A25e32cFC43De717950c9278,Aave Token,AAVE,18,0x4200000000000000000000000000000000000010,default,AAVE
5,10,0x764ad60e1b81f6cacfec1a2926393d688d4493e6,AladdinCRV,aCRV,18,0x4200000000000000000000000000000000000010,extended,aCRV
7,10,0xff733b2a3557a7ed6697007ab5d11b79fdd1b76b,Across Protocol Token,ACX,18,0x4200000000000000000000000000000000000010,extended,ACX
8,10,0x61BAADcF22d2565B0F471b291C475db5555e0b76,Aelin Protocol,AELIN,18,NaN,extended,AELIN
...,...,...,...,...,...,...,...,...
246,10,0x1F32b1c2345538c0c6f582fCB022739c4A194Ebb,Wrapped liquid staked Ether 2.0,wstETH,18,0x8E01013243a96601a86eb3153F0d9Fa4fbFb6957,default,wstETH
248,10,0xdb4eA87fF83eB1c80b8976FC47731Da6a31D35e5,wTBT(Bridge Token),wTBT,18,0x4200000000000000000000000000000000000010,extended,wTBT
250,10,0xE4F27b04cC7729901876B44f4EAA5102EC150265,CryptoFranc,XCHF,18,0x4200000000000000000000000000000000000010,extended,XCHF
252,10,0xD1917629B3E6A72E6772Aab5dBe58Eb7FA3C2F33,0x Protocol Token,ZRX,18,0x4200000000000000000000000000000000000010,default,ZRX


In [6]:
dune_list = dapi.get_dune_data(dune_list_query_id)
display(dune_list)

2023-04-27 12:50:46.503 | INFO     | duneapi_utils:get_dune_data:25 - Results available at https://dune.com/queries/2420477
2023-04-27 12:50:47,899 INFO dune_client.base_client waiting for query execution 01GZ1SZQPSG38BKBZ06HJ4Z36E to complete: ExecutionState.EXECUTING
2023-04-27 12:50:54.623 | INFO     | duneapi_utils:get_dune_data:40 - ✨ Results saved as csv_outputs/my_query_results.csv, with 371 rows and 7 columns.


,contract_address,decimals,is_counted_in_tvl,symbol,token_mapping_source,token_type,last_updated
0,0x298b9b95708152ff6968aafd889c6586e9169f1d,18,1,sBTC,manual,underlying,2023-04-27 16:50:47.129074+00:00
1,0xc84da6c8ec7a57cd10b939e79eaf9d2d17834e04,18,0,vUSD,manual,na,2023-04-27 16:50:47.129074+00:00
2,0xe4f27b04cc7729901876b44f4eaa5102ec150265,18,1,XCHF,manual,underlying,2023-04-27 16:50:47.129074+00:00
3,0x4e720dd3ac5cfe1e1fbde4935f386bb1c66f4642,18,1,BIFI,manual,underlying,2023-04-27 16:50:47.129074+00:00
4,0xb12a1be740b99d845af98098965af761be6bd7fe,18,0,CUSDCLP,manual,receipt,2023-04-27 16:50:47.129074+00:00
...,...,...,...,...,...,...,...
366,0xd3594e879b358f430e20f82bea61e83562d49d48,18,0,PSP,manual,receipt,2023-04-27 16:50:47.129074+00:00
367,0x1ccca1ce62c62f7be95d4a67722a8fdbed6eecb4,18,1,Multichain alETH,manual,underlying,2023-04-27 16:50:47.129074+00:00
368,0xb24f50dd9918934ab2228be7a097411ca28f6c14,18,0,vLUNA,manual,na,2023-04-27 16:50:47.129074+00:00
369,0x395ae52bb17aef68c2888d941736a71dc6d4e125,18,1,POOL,manual,underlying,2023-04-27 16:50:47.129074+00:00


In [11]:
#token addresses to lower
df['contract_address'] = df['address'].str.lower()
dune_list['contract_address'] = dune_list['contract_address'].str.lower()

In [12]:
adds = df[~df['contract_address'].isin(dune_list['contract_address'])]
display(adds[['address','symbol','decimals']])


,chainId,address,name,symbol,decimals,extensions_optimismBridgeAddress,extensions_opListId,extensions_opTokenId,contract_address
5,10,0x764ad60e1b81f6cacfec1a2926393d688d4493e6,AladdinCRV,aCRV,18,0x4200000000000000000000000000000000000010,extended,aCRV,0x764ad60e1b81f6cacfec1a2926393d688d4493e6
18,10,0xDb9888b842408B0b8eFa1f5477bD9F351754999E,BAXagent Coin,BAXA,18,0x4200000000000000000000000000000000000010,extended,BAXA,0xdb9888b842408b0b8efa1f5477bd9f351754999e
33,10,0xCa0E54b636DB823847B29F506BFFEE743F57729D,Chi USD,CHI,18,0x4200000000000000000000000000000000000010,extended,CHI,0xca0e54b636db823847b29f506bffee743f57729d
64,10,0x7b0bcC23851bBF7601efC9E9FE532Bf5284F65d3,Erica Social Token,EST,18,0x4200000000000000000000000000000000000010,extended,EST,0x7b0bcc23851bbf7601efc9e9fe532bf5284f65d3
120,10,0x3bB4445D30AC020a84c1b5A8A2C6248ebC9779D0,Theranos Coin,LIZ,18,0x4200000000000000000000000000000000000010,extended,LIZ,0x3bb4445d30ac020a84c1b5a8a2c6248ebc9779d0
122,10,0x0B3e851cf6508A16266BC68A651ea68b31ef673b,Loopfi,LPF,18,0x4200000000000000000000000000000000000010,extended,LPF,0x0b3e851cf6508a16266bc68a651ea68b31ef673b
134,10,0x77D40CBc27f912dcDbF4348cAf87B427c4D02486,Mochi,MOCHI,18,0x4200000000000000000000000000000000000010,extended,MOCHI,0x77d40cbc27f912dcdbf4348caf87b427c4d02486
138,10,0x5e70AfFE232e2919792f77EB94e566db0320fa88,Monetum,MOM,18,0x4200000000000000000000000000000000000010,extended,MOM,0x5e70affe232e2919792f77eb94e566db0320fa88
227,10,0x678d8f4Ba8DFE6bad51796351824DcCECeAefF2B,veKwenta,veKWENTA,18,0x4200000000000000000000000000000000000010,extended,veKWENTA,0x678d8f4ba8dfe6bad51796351824dcceceaeff2b
254,10,0x6e4cc0Ab2B4d2eDAfa6723cFA1582229F1Dd1Be1,Z.com USD,ZUSD,6,0x4200000000000000000000000000000000000010,extended,ZUSD,0x6e4cc0ab2b4d2edafa6723cfa1582229f1dd1be1
